In [2]:
import requests
import json
import pandas as pd

In [3]:
NOTION_API_KEY = "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"
NOTION_URL = "https://api.notion.com/v1/pages"
PAGE_ID = "15b2ec8473e98007885fe7ec343ff6df"

headers = {
    "Authorization": "Bearer " + NOTION_API_KEY,
    "Content-Type": "application/json", 
    "Notion-Version": "2022-06-28", 
}

# Page ID : 15b2ec8473e98007885fe7ec343ff6df
# DB ID: 15b2ec8473e9800ebac5e2e551a7727d

In [ ]:
#create


In [ ]:
#retrieve
def read_db():
    url = f"https://api.notion.com/v1/blocks/{PAGE_ID}/children"
    response = requests.get(url, headers=headers)

    data = response.json()
    result = data["results"]

    while data["has_more"]:
        next_url = url + f"?start_cursor={data['next_cursor']}"
        response = requests.get(next_url, headers=headers)
        data = response.json()  
        result += data["results"]

    if result == []: 
        return None
    return result

def display_db():
    res = read_db()
    if res == None:
        return None

    for i in len(res):
        print(res[i]["child_database"]["title"])

def display_customer_data(dbID : str):
    url = f"https://api.notion.com/v1/databases/{dbID}/query"
    response = requests.post(url, headers=headers)

    data = response.json()
    result = data["results"]

    while data["has_more"]:
        next_url = url + f"?start_cursor={data['next_cursor']}"
        response = requests.post(next_url, headers=headers)
        data = response.json()  
        result += data["results"]
    
    if result == []:
        return None
    return result

def get_customer_data(customer_ID : str):
    res = read_db()
    if res == None:
        return None

    for i in len(res):
        if res[i]["child_database"]["title"] == customer_ID:
            return display_customer_data(res[i]["child_database"]["id"])
    
    return None

    
    

In [ ]:
#update

In [ ]:
#delete

In [ ]:
#test

In [7]:
url = f"https://api.notion.com/v1/blocks/{PAGE_ID}/children"

response = requests.get(url, headers=headers)

data  = response.json()

with open("data.json", "w") as f:
    json.dump(data, f, indent=4)

res = data["results"]

while data["has_more"]:
    next_url = f"{url}?start_cursor={data['next_cursor']}"
    response = requests.get(next_url, headers=headers)
    data = response.json()
    res.extend(data["results"])